In [1]:
%%time

import numpy as np
import torch
import gc
import random
from torch import nn
from sklearn.metrics import mean_absolute_error
from tqdm.notebook import tqdm
from NN_models import NN_2_256, NN_4_256, NN_8_256, NN_8_64, test # NN_8_128
from reaction_energy_calculation import calculate_reaction_energy
from prepare_data import prepare, save_chk, load_chk


def set_random_seed(seed):
    # seed everything
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(41)

CPU times: user 2.67 s, sys: 1.61 s, total: 4.28 s
Wall time: 26.5 s


In [2]:
# %%time
# data, data_train, data_test = prepare(path='data', test_size=0.2)

In [3]:
# %%time
# save_chk(data, data_train, data_test, path='checkpoints')

In [2]:
%%time
data, data_train, data_test = load_chk(path='checkpoints')

CPU times: user 5.1 s, sys: 5.9 s, total: 11 s
Wall time: 17.3 s


In [7]:
data[73] # take the easiest reaction H2 = 2H

{'Database': 'MGAE109',
 'Components': array(['H_mgae109', 'H2_mgae109'], dtype='<U20'),
 'Coefficients': tensor([ 2., -1.]),
 'Energy': tensor(109.4900),
 'Grid': tensor([[ -1.1678, -16.1181, -16.0818,  ..., -16.1181, -16.1033, -16.1181],
         [ -1.1678, -16.1181, -14.7017,  ..., -16.1181, -15.3051, -16.1181],
         [ -1.1678, -16.1181, -12.3610,  ..., -16.1181, -13.2385, -16.1181],
         ...,
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148],
         [ -7.7005,  -7.7005, -13.7295,  ..., -13.7295,  -8.2041,  -8.2041],
         [ -8.0004,  -8.0004, -14.2658,  ..., -14.2658,  -8.5148,  -8.5148]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.6055e-02, 2.2680e-02,
         2.6055e-02]),
 'Densities': tensor([[0.3111, 0.0000],
         [0.3111, 0.0000],
         [0.3111, 0.0000],
         ...,
         [0.0003, 0.0003],
         [0.0005, 0.0005],
         [0.0003, 0.0003]]),
 'Gradients': tensor([[3.6975e-09, 0.0000e+00, 0.0000e+00],

In [8]:
data_test[0]

{'Database': 'NCCE31',
 'Components': array(['HCl-HCl_ncce31', 'HCl_ncce31'], dtype='<U20'),
 'Coefficients': tensor([-1.,  2.]),
 'Energy': tensor(2.0100),
 'Grid': tensor([[1.8194, 1.8079, 1.9051,  ..., 1.9412, 1.8152, 1.8091],
         [1.8193, 1.8079, 2.0156,  ..., 2.0500, 1.8210, 1.8145],
         [1.8193, 1.8079, 2.0797,  ..., 2.1131, 1.8581, 1.8490],
         ...,
         [1.1998, 1.2390, 1.2328,  ..., 1.2794, 1.1804, 1.2182],
         [1.2114, 1.2497, 1.2454,  ..., 1.2917, 1.1918, 1.2288],
         [1.1998, 1.2390, 1.2328,  ..., 1.2794, 1.1804, 1.2182]]),
 'Weights': tensor([2.5717e-17, 9.9780e-15, 3.2611e-13,  ..., 2.5979e-02, 2.2624e-02,
         2.5979e-02]),
 'Densities': tensor([[1.5987e+03, 1.5987e+03],
         [1.5986e+03, 1.5986e+03],
         [1.5977e+03, 1.5977e+03],
         ...,
         [2.7319e-04, 2.7319e-04],
         [3.6636e-04, 3.6636e-04],
         [2.7319e-04, 2.7319e-04]]),
 'Gradients': tensor([[2.1993e+05, 2.1993e+05, 2.1993e+05],
         [1.8339e+07,

In [9]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):

        self.data = data
        
    def __getitem__(self, i):
        # i = 1
        self.data[i].pop('Database', None)

        return self.data[i], self.data[i]['Energy']
    
    def __len__(self):
        return len(self.data.keys())


train_set = Dataset(data=data_train)
train_dataloader = torch.utils.data.DataLoader(train_set, 
                                               batch_size=None,
                                               num_workers=1,
                                               pin_memory=True,
                                               shuffle=True)

test_set = Dataset(data=data_test)
test_dataloader = torch.utils.data.DataLoader(test_set, 
                                              batch_size=None,
                                              num_workers=1,
                                              pin_memory=True,
                                              shuffle=True)

In [6]:
import NN_models
reload(NN_models)

NameError: name 'reload' is not defined

In [10]:
del optimizer
del model

NameError: name 'optimizer' is not defined

In [11]:
device = torch.device('cuda:0') if torch.cuda.is_available else torch.device('cpu')
print(device)
# model = NN_2_256(nconstants=21, DFT='SVWN').to(device)

model = NN_4_256(nconstants=24, DFT='PBE').to(device)

model.load_state_dict(torch.load('model_chk/predopt/PBE_4_256.param', map_location=device))

cuda:0


<All keys matched successfully>

In [12]:
criterion = nn.MSELoss()

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

In [249]:
import torch_optimizer as optim2


optimizer = optim2.RAdam(
    model.parameters(),
    lr= 3e-4,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0,
)

In [241]:
optimizer = optim2.QHAdam(
    model.parameters(),
    lr= 1e-4,
    betas=(0.9, 0.999),
    nus=(1.0, 1.0),
    weight_decay=0,
    decouple_weight_decay=False,
    eps=1e-8,
)

In [15]:
from importlib import reload
import predopt
reload(predopt)

<module 'predopt' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/predopt.py'>

In [16]:
%%time

from predopt import DatasetPredopt
train_predopt_set = DatasetPredopt(data=data, dft='PBE')
train_predopt_dataloader = torch.utils.data.DataLoader(train_predopt_set,
                                                       batch_size=None,
                                                       num_workers=1,
                                                       pin_memory=True,
                                                       shuffle=True)

CPU times: user 134 µs, sys: 0 ns, total: 134 µs
Wall time: 141 µs


In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999), weight_decay=0.01)

In [18]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=5e-3)

In [21]:
from predopt import predopt


predopt(model, criterion, optimizer, train_predopt_dataloader, device, n_epochs=10, accum_iter=10)

Epoch 1


  0%|          | 0/267 [00:00<?, ?it/s]

ValueError: Input contains NaN.

In [350]:
# torch.save(model.state_dict(), 'model_chk/predopt/PBE_4_256.param')

In [11]:
torch.cuda.empty_cache()

In [184]:
from tqdm.notebook import tqdm
import os

log_file_path = 'log/epoch_training.log'

if os.path.isfile(log_file_path):
    os.remove(log_file_path)

In [360]:
def train(model, criterion, optimizer, train_loader, test_loader, n_epochs=20, accum_iter=4, verbose=False):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        model.train()
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for batch_idx, (X_batch, y_batch) in enumerate(progress_bar_train):
            
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            predictions = model(X_batch_grid)
            
            # print(predictions)
            # predictions = torch.tile(torch.Tensor(true_constants_PBE), [X_batch_grid.shape[0],1]).to(device, non_blocking=True)
            # print(predictions)
            # print(torch.max(torch.abs(predictions)))
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')
            # print(reaction_energy)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f}")
            loss = criterion(reaction_energy, y_batch)

            # loss_accumulation
            loss = loss / accum_iter
            
            loss.backward()
            
            if ((batch_idx + 1) % accum_iter == 0) or (batch_idx + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)

                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                train_mse_losses_per_epoch.append(MSE)
                train_mae_losses_per_epoch.append(MAE)
                progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)

                
                with open(log_file_path, 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                    # f.write(f"{preds}\n")
                    
                del MAE, MSE
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy
            gc.collect()
            torch.cuda.empty_cache()
            # gpu_usage()
        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        progress_bar_test = tqdm(test_loader)
        model.eval()
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        with torch.no_grad():
            for X_batch, y_batch in progress_bar_test:
                X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
                # print(f"{X_batch['Components']}")
                predictions = model(X_batch_grid)
                reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung='GGA', dft='PBE')
                loss = criterion(reaction_energy, y_batch)
                MSE = loss.item()
                MAE = torch.abs(reaction_energy - y_batch).item()
                progress_bar_train.set_postfix(MAE = MAE)
                test_mse_losses_per_epoch.append(MSE)
                test_mae_losses_per_epoch.append(MAE)
                
                with open(log_file_path, 'a') as f:
                    f.write(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}\n")
                del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
                gc.collect()
                torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [361]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [353]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.999))

In [362]:
train_loss_mae = train(model, criterion, optimizer,
                              train_dataloader,
                              test_dataloader, n_epochs=10)

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 1.09 GiB (GPU 0; 31.75 GiB total capacity; 29.18 GiB already allocated; 1.08 GiB free; 29.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
train_loss_mae

In [261]:
def test_dft(train_loader, test_loader, c_arr, criterion, rung, dft, verbose=False, n_epochs=1, model_eval=False):
    train_loss_mae = []
    train_loss_mse = []
    test_loss_mae = []
    test_loss_mse = []

    for epoch in range(n_epochs):
        print(f'Epoch {epoch+1}')
        # train
        
        progress_bar_train = tqdm(train_loader)
        train_mae_losses_per_epoch = []
        train_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_train:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            # print(torch.tile(c_arr, [X_batch_grid.shape[0],1]))
            if model_eval:
                predictions = model(X_batch_grid)
            else:
                predictions = torch.tile(c_arr, [X_batch_grid.shape[0],1]).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            train_mse_losses_per_epoch.append(MSE)
            train_mae_losses_per_epoch.append(MAE)
            progress_bar_train.set_postfix(MSE = MSE, MAE = MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")

            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()

        
        train_loss_mse.append(np.mean(train_mse_losses_per_epoch))        
        train_loss_mae.append(np.mean(train_mae_losses_per_epoch))

        print(f'train MSE Loss = {train_loss_mse[epoch]:.8f} MAE Loss = {train_loss_mae[epoch]:.8f}')


        
        #test
        progress_bar_test = tqdm(test_loader)
        test_mae_losses_per_epoch = []
        test_mse_losses_per_epoch = []
        for X_batch, y_batch in progress_bar_test:
            # print(f"{X_batch['Components']}")
            X_batch_grid, y_batch = X_batch['Grid'].to(device), y_batch.to(device)
            if model_eval:
                predictions = model(X_batch_grid)
            else:
                predictions = torch.tile(c_arr, [X_batch_grid.shape[0],1]).to(device)
            reaction_energy = calculate_reaction_energy(X_batch, predictions, device, rung, dft)
            loss = criterion(reaction_energy, y_batch)
            MSE = loss.item()
            MAE = torch.abs(reaction_energy - y_batch).item()
            # progress_bar_train.set_postfix(MAE = MAE)
            test_mse_losses_per_epoch.append(MSE)
            test_mae_losses_per_epoch.append(MAE)
            if verbose:
                print(f"{X_batch['Components']} pred {reaction_energy.item():4f} true {y_batch.item():4f} MSE {MSE:4f} MAE {MAE:4f}")
            
            del X_batch, X_batch_grid, y_batch, predictions, reaction_energy, loss, MAE, MSE
            gc.collect()
            torch.cuda.empty_cache()
                
        test_loss_mse.append(np.mean(test_mse_losses_per_epoch))
        test_loss_mae.append(np.mean(test_mae_losses_per_epoch))

        print(f'test MSE Loss = {test_loss_mse[epoch]:.8f} MAE Loss = {test_loss_mae[epoch]:.8f}')

    return train_loss_mae

In [262]:
from importlib import reload
import NN_models
reload(NN_models)
from NN_models import NN_2_256, NN_8_256, NN_8_64
# import SVWN3

In [263]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [342]:
true_constants_SVWN = torch.Tensor([0.0310907, 0.01554535, 
                3.72744,   7.06042,
                12.9352,   18.0578,
                -0.10498,  -0.32500,
                0.0310907,  0.01554535,  -1/(6*torch.pi**2),
                13.0720,    20.1231,      1.06835,
                42.7198,   101.578,      11.4813,
                -0.409286,  -0.743294,   -0.228344,
                1])

true_constants_PBE = torch.Tensor([[0.06672455,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709921,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 3.3662,  0.88026,
       0.49294, 0.62517, 0.49671,
       # 1,  1,  1,
       0.031091, 0.015545, 0.016887,
       0.21370,  0.20548,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])

par_rpw92_constants = torch.Tensor([[0.06672455,
       (1 - torch.log(torch.Tensor([2])))/(torch.pi**2),
       1.709920934161365617563962776245,
       7.5957, 14.1189, 10.357,
       3.5876, 6.1977, 3.6231,
       1.6382, 4.86059,  0.88026,
       0.49294, 0.750188, 0.49671,
       # 1,  1,  1,
       0.0310907, 0.01554535, 0.0168869,
       0.21370,  0.266529,  0.11125,
       -3/8*(3/torch.pi)**(1/3)*4**(2/3),
       0.8040,
       0.2195149727645171]])


# true_constants

test_dft(train_dataloader, test_dataloader, true_constants_PBE, criterion, rung='GGA', dft='PBE', verbose=False, model_eval=False)
# test_dft(train_dataloader, test_dataloader, true_constants_SVWN, criterion, rung='LDA', dft='SVWN3', verbose=False)

# broken_PBE 45.47375717 35.76542641
# Mean Absolute Error, kcal/mol
# DFT              train        test
# SVWN3            32.63222432 26.81526929
# PBE_sigma_broken 7.84055060  7.63013082
# PBE_fixed        7.85833597  7.64929252
# PBE_pred_4_256      

Epoch 1


  0%|          | 0/215 [00:00<?, ?it/s]

train MSE Loss = 115.08597731 MAE Loss = 7.85833597


  0%|          | 0/52 [00:00<?, ?it/s]

test MSE Loss = 108.85051548 MAE Loss = 7.64929252


[7.858335966790138]

tensor([[4.4521661475300788879394531e-03, 9.6663110889494419097900391e-04,
         2.9238297939300537109375000e+00, 5.7694656372070312500000000e+01,
         1.9934335327148437500000000e+02, 1.0726745605468750000000000e+02,
         1.2870873451232910156250000e+01, 3.8411483764648437500000000e+01,
         1.3126853942871093750000000e+01, 2.6836993694305419921875000e+00,
         1.1331302642822265625000000e+01, 7.7485764026641845703125000e-01,
         2.4298985302448272705078125e-01, 3.9083752036094665527343750e-01,
         2.4672082066535949707031250e-01, 9.6665031742304563522338867e-04,
         2.4164703791029751300811768e-04, 2.8517076862044632434844971e-04,
         4.5667689293622970581054688e-02, 4.2222026735544204711914062e-02,
         1.2376561760902404785156250e-02, 8.6587810516357421875000000e-01,
         6.4641600847244262695312500e-01, 4.8186820000410079956054688e-02]])

In [16]:
from importlib import reload
import PBE
import reaction_energy_calculation
import utils
import NN_models
reload(NN_models)
reload(utils)
reload(reaction_energy_calculation)
reload(PBE)
from reaction_energy_calculation import calculate_reaction_energy

In [117]:
from importlib import reload
import PBE
reload(PBE)
# from PBE_new import F_PBE_new

<module 'PBE' from '/home/duzaripov/ML-parameterization-of-DFT-functionals/PBE.py'>

In [35]:
calculate_reaction_energy(data[73], torch.tile(true_constants_PBE, [data[73]['Grid'].shape[0],1]).to(device='cpu'), device='cpu', rung='GGA', dft='PBE')

tensor(104.6450347900390625000000000)

In [27]:
c_arr = torch.load('log/c_arr.pt')
c_arr

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0',
       requires_grad=True)

In [33]:
res_fh = torch.load('log/res_fH.pt')
res_fh

tensor([7.5301858259990694932639599e-07, 5.7899451348930597305297852e-05,
        1.0916657047346234321594238e-03,  ...,
        1.9441943615674972534179688e-02, 4.2287241667509078979492188e-02,
        4.4258534908294677734375000e-02], device='cuda:0', requires_grad=True)

In [151]:
log = torch.load('log/log.pt')
log[log.isnan()]

tensor([nan], device='cuda:0', grad_fn=<IndexBackward0>)

In [162]:
f2_ = torch.load('log/f2_.pt')
f2_[f2_ == -1] + eps

tensor([-0.9999998807907104492187500, -0.9999998807907104492187500],
       device='cuda:0', grad_fn=<AddBackward0>)

In [169]:
log_calc = torch.where(f2_ < -1, torch.log1p(f2_ + eps), torch.log1p(f2_))


In [170]:
f2_[f2_ < -1]

tensor([-1.0000001192092895507812500], device='cuda:0',
       grad_fn=<IndexBackward0>)

In [171]:
log_calc.isnan().sum()

tensor(0, device='cuda:0')

In [86]:
log.median()

tensor(0.1908233761787414550781250, device='cuda:0', grad_fn=<MedianBackward0>)

In [111]:
eps = 10e-8

In [157]:
torch.log1p(torch.Tensor([-1+eps]))

tensor([-15.9423847198486328125000000])

In [110]:
x = torch.expm1(log[log.isinf().logical_not()])

In [97]:
log[log.isinf().logical_not()].shape

torch.Size([393239])

In [101]:
x

tensor([2.7483220037538558244705200e-05, 2.4236764293164014816284180e-03,
        7.8470092266798019409179688e-03,  ...,
        1.2916454672813415527343750e-01, 2.0304933190345764160156250e-01,
        4.1896623373031616210937500e-01], device='cuda:0',
       grad_fn=<Expm1Backward0>)

In [99]:
log

tensor([2.7482841687742620706558228e-05, 2.4207441601902246475219727e-03,
        7.8163817524909973144531250e-03,  ...,
        1.2147802114486694335937500e-01, 1.8485945463180541992187500e-01,
        3.4992858767509460449218750e-01], device='cuda:0', requires_grad=True)

In [107]:
torch.log1p(x+10e-9)

tensor([2.7492842491483315825462341e-05, 2.4207539390772581100463867e-03,
        7.8163919970393180847167969e-03,  ...,
        1.2147803604602813720703125e-01, 1.8485945463180541992187500e-01,
        3.4992858767509460449218750e-01], device='cuda:0',
       grad_fn=<Log1PBackward0>)

In [115]:
abs(x)[x > 0].min()

tensor(1.4420080407528956811802345e-10, device='cuda:0',
       grad_fn=<MinBackward1>)

In [62]:
torch.log1p(torch.nan_to_num(torch.Tensor([torch.inf])))

tensor([88.7228393554687500000000000])

In [131]:
f2_ = torch.Tensor([1, 2, 3, -1])

In [132]:
torch.where(f2_ == -1, torch.log1p(f2_ + eps), torch.log1p(f2_))

tensor([  0.6931471824645996093750000,   1.0986123085021972656250000,
          1.3862943649291992187500000, -15.9423847198486328125000000])

In [134]:
res_A = torch.load('log/res_A.pt')
res_A

tensor([1.9993112569723063976900174e-18, 5.8318030834197998046875000e-01,
        9.9735140800476074218750000e-01,  ...,
        3.4186925888061523437500000e+00, 2.2988363343756645917892456e-04,
        4.0849060058593750000000000e+01], device='cuda:0', requires_grad=True)

In [141]:
res_A[res_A < 0]

tensor([-1358.8190917968750000000000000, -1140.9973144531250000000000000,
        -1358.6059570312500000000000000,  ...,
         -931.7769775390625000000000000,  -751.9721069335937500000000000,
         -994.9572143554687500000000000], device='cuda:0',
       grad_fn=<IndexBackward0>)

In [142]:
(1 - torch.log(torch.Tensor([2])))/(torch.pi**2)

tensor([0.0310906916856765747070312])